In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Data_prepare.schemas import gen_stats_light_schema
from Data_prepare.query_prepare_BayesCard import prepare_join_queries
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.Bayescard_BN import Bayescard_BN, build_meta_info
from Models.BN_ensemble_model import BN_ensemble
model_path = "/home/ziniu.wzn/stats/BayesCard_ensemble/naive_ensemble.pkl"
with open(model_path, "rb") as f:
    naive_ensemble = pickle.load(f)

In [ ]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Structure.StatisticalTypes import MetaType
from Data_prepare.join_data_preparation import JoinDataPreparator
from Data_prepare.schemas import gen_stats_light_schema
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)


sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.Bayescard_BN import Bayescard_BN, build_meta_info
from Models.BN_ensemble_model import BN_ensemble
data_folder = "/home/ziniu.wzn/stats/gen_single_light/"
naive_ensemble = BN_ensemble(schema)
naive_ensemble.join_prepare = prep
for table_name in ["posts", "users", "postLinks", "postHistory", "votes", "comments", "badges", "tags"]:
    df = pd.read_hdf(data_folder+table_name+".hdf")
    new_cols = []
    for col in df.columns:
        if "PostId" in col or "UserId" in col or "mul" in col:
            print(col)
            df = df.drop(col, axis=1)
        else:
            new_cols.append(col.split(".")[-1])
    df.columns = new_cols
    bn = Bayescard_BN(table_name, nrows=len(df))
    bn.build_from_data(df, algorithm="chow-liu", max_parents=1, ignore_cols=['Id'], sample_size=200000)
    bn.infer_algo = "exact-jit"
    bn.init_inference_method()
    naive_ensemble.bns[table_name] = bn
model_path = "/home/ziniu.wzn/stats/BayesCard_ensemble/naive_ensemble.pkl"
pickle.dump(naive_ensemble, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

In [5]:
sys.path.append('/home/ziniu.wzn/deepdb-stats')
from evaluation.utils import parse_query
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
with open(query_filename, "r") as f:
    real_query = f.readlines()

In [6]:
import time
latency = []
q_errors = []
pred_all = []
for i, q in enumerate(real_query):
    true = int(q.split("||")[-1])
    query = parse_query(q.split("||")[0], schema)
    tic = time.time()
    try:
        pred = naive_ensemble.naive_cardinality(query)
        if type(pred) == list or type(pred) == np.ndarray:
            pred = pred[0]
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        pred = None
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        print(i)
        pred = 1
    pred_all.append(pred)
    error = max(pred / true, true / pred)
    #print(f"predicting query no {i}: {real_query[i]} \n")
    #print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

q-error 50% percentile is 3.5244923746587333
q-error 90% percentile is 79.59394307186865
q-error 95% percentile is 295.25225328628005
q-error 99% percentile is 22138.387329062032
q-error 100% percentile is 27819576.665189616
average latency is 6.597885204378202 ms


In [7]:
np.save("/home/ziniu.wzn/stats/est_results/nativeBayes_join", pred_all)

In [ ]:
print(q.table_set)
print(q.table_where_condition_dict)

In [ ]:
naive_ensemble.bns["posts"].encoding

In [ ]:
data_folder = "/home/ziniu.wzn/stats/gen_single_light/"
df = pd.read_hdf(data_folder+"posts.hdf")

In [ ]:
df.columns

In [ ]:
bn.encoding